In [1]:
import pandas as pd

### Westernized samples from [curatedMetagenomicsData](http://bioconductor.org/packages/release/data/experiment/html/curatedMetagenomicData.html) package

In [2]:
west_count = pd.read_csv("curated_metagenomics/data/westernized_count.csv", index_col=0)

Choosing randomly 100 samples

In [3]:
west_count = west_count.sample(100, axis=1)

In [4]:
west_count

,CosteaPI_2017.metaphlan_bugs_list.stool:SID713B029-11-0-0,NielsenHB_2014.metaphlan_bugs_list.stool:MH0009,VatanenT_2016.metaphlan_bugs_list.stool:G78696,QinN_2014.metaphlan_bugs_list.stool:LD-17,HansenLBS_2018.metaphlan_bugs_list.stool:SID5465-1,ZeeviD_2015.metaphlan_bugs_list.stool:PNP_Main_568,SchirmerM_2016.metaphlan_bugs_list.stool:G88782,VatanenT_2016.metaphlan_bugs_list.stool:G80388,JieZ_2017.metaphlan_bugs_list.stool:SAMEA104142374,HansenLBS_2018.metaphlan_bugs_list.stool:SID5469-3,...,DhakanDB_2019.metaphlan_bugs_list.stool:DHAK_C3_M,LiJ_2014.metaphlan_bugs_list.stool:MH0388,HansenLBS_2018.metaphlan_bugs_list.stool:SID5453-3,ZeeviD_2015.metaphlan_bugs_list.stool:PNP_Main_75,ThomasAM_2018b.metaphlan_bugs_list.stool:CRC_MR_SBJ07C_17,HMP_2012.metaphlan_bugs_list.stool:SRS018656,CosteaPI_2017.metaphlan_bugs_list.stool:halbarad-11-7-0,HanniganGD_2017.metaphlan_bugs_list.stool:MG100158,ZeeviD_2015.metaphlan_bugs_list.stool:PNP_Main_15,QinJ_2012.metaphlan_bugs_list.stool:NOM014
k__Bacteria,63610183,56873393,17782886,99553661,67440742,24921602,29218063,16336154,35707838,61980614,...,10099415,61784710,88346492,30770811,109631013,50514047,87366861,7233984,40792093,32125392
k__Bacteria|p__Bacteroidetes,37819577,11263353,616851,65472532,1369024,20727994,5816542,12153082,21703113,12457219,...,6505673,42003087,22913272,9508965,40760413,22574308,20097999,1104902,16061729,27544221
k__Bacteria|p__Proteobacteria,6362375,1176865,68706,8665996,48091,349396,65165,150497,560745,606306,...,265566,1432514,529747,184521,1201162,64522,4290480,625147,902873,389087
k__Bacteria|p__Firmicutes,18612306,43332477,3795599,25282415,63265903,3441078,21250341,2477539,13342316,44509708,...,3301467,16420064,62414587,17715172,50499970,26321506,41799407,5325422,20584491,3593585
k__Bacteria|p__Actinobacteria,815925,207139,6782686,127278,1481885,354385,2086015,1555036,101664,4399930,...,26710,1927419,2344840,3349952,11702997,1444197,8332080,174935,1124896,352795
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
k__Archaea|p__Euryarchaeota|c__Methanobacteria|o__Methanobacteriales|f__Methanobacteriaceae|g__Methanobacterium|s__Methanobacterium_unclassified,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
k__Viruses|p__Viruses_noname|c__Viruses_noname|o__Caudovirales|f__Podoviridae|g__Podoviridae_noname|s__Clostridium_phage_phiCP7R,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
k__Viruses|p__Viruses_noname|c__Viruses_noname|o__Caudovirales|f__Podoviridae|g__Podoviridae_noname|s__Clostridium_phage_phiCP7R|t__PRJNA167663,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
k__Bacteria|p__Firmicutes|c__Bacilli|o__Bacillales|f__Staphylococcaceae|g__Staphylococcus|s__Staphylococcus_vitulinus,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Non-westernized samples from [curatedMetagenomicsData](http://bioconductor.org/packages/release/data/experiment/html/curatedMetagenomicData.html) package

In [5]:
non_west_count = pd.read_csv("curated_metagenomics/data/non_westernized_count.csv", index_col=0)

In [6]:
non_west_count = non_west_count.sample(100, axis=1)

### Sediment samples from Archaelogical sediments

In [8]:
sediment_count = pd.read_csv("PRJEB18629/data/sediment_count.csv", index_col=1).drop("Unnamed: 0", 1)

### Marsha's samples

In [9]:
import os

In [10]:
def get_samp_basename(samp_name):
    return("_".join(samp_name.split("_")[0:2]))

In [11]:
def read_sample(samp_name):
    return(pd.read_csv("wibowo_mp/data/"+samp_name, 
                       sep="\t", index_col="#SampleID")
           .rename(columns={'Metaphlan2_Analysis':get_samp_basename(samp_name)}))

In [12]:
soil_samp = ['Lib4_10','Lib4_11','Lib4_12']

In [13]:
wib_samp = [i for i in os.listdir("wibowo_mp/data/") if i.startswith("Lib")]

In [14]:
wib = read_sample(wib_samp[0])

In [15]:
for i in wib_samp[1:]:
    wib = wib.merge(read_sample(i), left_index=True, right_index=True, how='outer')

In [16]:
wib = wib.fillna(0)

In [52]:
wib

,Lib3_6,Lib4_2,Lib4_1,Lib4_12,Lib4_7,Lib4_4,Lib4_11,Lib4_8,Lib4_5,Lib4_10,Lib4_6,Lib3_9,Lib4_3
#SampleID,,,,,,,,,,,,,
k__Archaea,0.00666,0.10627,3.52607,0.0049,0.02790,0.47962,0.0,0.03779,0.02775,0.0,0.52044,7.18607,0.32794
k__Archaea|p__Euryarchaeota,0.00666,0.10627,3.52607,0.0049,0.02790,0.47962,0.0,0.03779,0.02775,0.0,0.52044,7.18607,0.32794
k__Archaea|p__Euryarchaeota|c__Halobacteria,0.00000,0.00000,0.00000,0.0000,0.00680,0.00000,0.0,0.03779,0.00000,0.0,0.00000,0.00000,0.00000
k__Archaea|p__Euryarchaeota|c__Halobacteria|o__Halobacteriales,0.00000,0.00000,0.00000,0.0000,0.00680,0.00000,0.0,0.03779,0.00000,0.0,0.00000,0.00000,0.00000
k__Archaea|p__Euryarchaeota|c__Halobacteria|o__Halobacteriales|f__Halobacteriaceae,0.00000,0.00000,0.00000,0.0000,0.00680,0.00000,0.0,0.03779,0.00000,0.0,0.00000,0.00000,0.00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
k__Viruses|p__Viruses_noname|c__Viruses_noname|o__Viruses_noname|f__Inoviridae|g__Plectrovirus|s__Plectrovirus_unclassified,0.00000,0.00000,0.00000,0.0000,0.00456,0.00000,0.0,0.00000,0.00000,0.0,0.00000,0.00000,0.00000
k__Viruses|p__Viruses_noname|c__Viruses_noname|o__Viruses_noname|f__Parvoviridae,0.00000,0.00000,0.00000,0.0000,0.00000,0.46174,0.0,0.00000,0.00000,0.0,0.00000,0.00000,0.00000
k__Viruses|p__Viruses_noname|c__Viruses_noname|o__Viruses_noname|f__Parvoviridae|g__Bocavirus,0.00000,0.00000,0.00000,0.0000,0.00000,0.46174,0.0,0.00000,0.00000,0.0,0.00000,0.00000,0.00000


### Merging all samples

In [17]:
all_samp = (west_count.merge(non_west_count, left_index=True, right_index=True, how='outer')
            .merge(sediment_count, left_index=True, right_index=True, how='outer')
            .merge(wib, left_index=True, right_index=True, how='outer').fillna(0))

Keeping on Taxons that occur in more than 10% of samples

In [18]:
all_samp = all_samp.loc[all_samp.astype(bool).sum(axis=1) >= int(all_samp.shape[1]/10),:]

In [19]:
all_samp

,CosteaPI_2017.metaphlan_bugs_list.stool:SID713B029-11-0-0,NielsenHB_2014.metaphlan_bugs_list.stool:MH0009,VatanenT_2016.metaphlan_bugs_list.stool:G78696,QinN_2014.metaphlan_bugs_list.stool:LD-17,HansenLBS_2018.metaphlan_bugs_list.stool:SID5465-1,ZeeviD_2015.metaphlan_bugs_list.stool:PNP_Main_568,SchirmerM_2016.metaphlan_bugs_list.stool:G88782,VatanenT_2016.metaphlan_bugs_list.stool:G80388,JieZ_2017.metaphlan_bugs_list.stool:SAMEA104142374,HansenLBS_2018.metaphlan_bugs_list.stool:SID5469-3,...,Lib4_12,Lib4_7,Lib4_4,Lib4_11,Lib4_8,Lib4_5,Lib4_10,Lib4_6,Lib3_9,Lib4_3
k__Archaea,0.0,160961.0,0.0,69457.0,5803194.0,0.0,0.0,0.0,0.0,0.0,...,0.0049,0.0279,0.47962,0.0,0.03779,0.02775,0.0,0.52044,7.18607,0.32794
k__Archaea|p__Euryarchaeota,0.0,160961.0,0.0,69457.0,5803194.0,0.0,0.0,0.0,0.0,0.0,...,0.0049,0.0279,0.47962,0.0,0.03779,0.02775,0.0,0.52044,7.18607,0.32794
k__Archaea|p__Euryarchaeota|c__Methanobacteria,0.0,160961.0,0.0,69457.0,5803194.0,0.0,0.0,0.0,0.0,0.0,...,0.0049,0.0211,0.47962,0.0,0.00000,0.02775,0.0,0.52044,7.18607,0.32794
k__Archaea|p__Euryarchaeota|c__Methanobacteria|o__Methanobacteriales,0.0,160961.0,0.0,69457.0,5803194.0,0.0,0.0,0.0,0.0,0.0,...,0.0049,0.0211,0.47962,0.0,0.00000,0.02775,0.0,0.52044,7.18607,0.32794
k__Archaea|p__Euryarchaeota|c__Methanobacteria|o__Methanobacteriales|f__Methanobacteriaceae,0.0,160961.0,0.0,69457.0,5803194.0,0.0,0.0,0.0,0.0,0.0,...,0.0049,0.0211,0.47962,0.0,0.00000,0.02775,0.0,0.52044,7.18607,0.32794
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
k__Viruses|p__Viruses_noname|c__Viruses_noname|o__Viruses_noname|f__Potyviridae,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0000,0.0000,0.00000,0.0,0.00000,0.00000,0.0,0.00000,0.00000,0.00000
k__Viruses|p__Viruses_noname|c__Viruses_noname|o__Viruses_noname|f__Potyviridae|g__Potyvirus,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0000,0.0000,0.00000,0.0,0.00000,0.00000,0.0,0.00000,0.00000,0.00000
k__Viruses|p__Viruses_noname|c__Viruses_noname|o__Viruses_noname|f__Potyviridae|g__Potyvirus|s__Dasheen_mosaic_virus,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0000,0.0000,0.00000,0.0,0.00000,0.00000,0.0,0.00000,0.00000,0.00000
k__Viruses|p__Viruses_noname|c__Viruses_noname|o__Viruses_noname|f__Potyviridae|g__Potyvirus|s__Dasheen_mosaic_virus|t__PRJNA15388,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0000,0.0000,0.00000,0.0,0.00000,0.00000,0.0,0.00000,0.00000,0.00000


### Preparing metadata 

In [30]:
meta_west = pd.Series(index=west_count.columns, data=['westernized_humans']*west_count.shape[1]).to_frame(name='Env')
meta_west['SourceSink'] = ["source"]*west_count.shape[1]

In [33]:
meta_non_west = pd.Series(index=non_west_count.columns, data=['non_westernized_humans']*non_west_count.shape[1]).to_frame(name='Env')
meta_non_west['SourceSink'] = ["source"]*non_west_count.shape[1]

In [35]:
meta_sediment = pd.Series(index=sediment_count.columns, data=['archeo_sediment']*sediment_count.shape[1]).to_frame(name='Env')
meta_sediment['SourceSink'] = ["source"]*sediment_count.shape[1]

In [50]:
meta_wib_sink = pd.Series(index=wib.columns, data=["-"]*wib.shape[1]).to_frame(name='Env')
meta_wib_sink['SourceSink'] = ['sink']*wib.shape[1]
meta_wib_sink.drop(soil_samp, axis=0, inplace=True)

In [57]:
meta_wib_source = pd.Series(index=soil_samp, data=['archeo_sediment']*len(soil_samp)).to_frame(name="Env")
meta_wib_source['SourceSink'] = ['source']*len(soil_samp)

In [58]:
meta = meta_west.append(meta_non_west).append(meta_wib_sink).append(meta_wib_source)

In [60]:
meta = meta[['SourceSink','Env']]

In [61]:
meta

,SourceSink,Env
CosteaPI_2017.metaphlan_bugs_list.stool:SID713B029-11-0-0,source,westernized_humans
NielsenHB_2014.metaphlan_bugs_list.stool:MH0009,source,westernized_humans
VatanenT_2016.metaphlan_bugs_list.stool:G78696,source,westernized_humans
QinN_2014.metaphlan_bugs_list.stool:LD-17,source,westernized_humans
HansenLBS_2018.metaphlan_bugs_list.stool:SID5465-1,source,westernized_humans
...,...,...
Lib3_9,sink,-
Lib4_3,sink,-
Lib4_10,source,archeo_sediment
Lib4_11,source,archeo_sediment


### Exporting to csv

In [62]:
all_samp.to_csv("all_samp_st2.tsv", sep="\t", index_label='taxon')

In [63]:
meta.to_csv("labels_st2.tsv", sep="\t", index_label='#SampleID')

### Converting counts file to biom format

In [64]:
!biom convert -i all_samp_st2.tsv -o all_samp_st2.biom --table-type="Taxon table" --to-json